## Exemplo 02. O problema de transporte

### Custos de transporte por tonelada, produção e Demanda

| SID/FAB|Farmington [$-Ton]|Detroit [$-Ton]|Lansing [$-Ton]|Winsor [$-Ton]| St. Louis [$-Ton] | Fremont [$-Ton]| Lafayette [$-Ton]| **Produção [Ton]**|
|:----:|:----:|:----:|:----:|:----:|:----:|:----:|:----:| :----:|
|Gary| 39 | 14 | 11 | 14 | 16 | 82 | 8 | 1400 |
|Cleveland| 27 | 9 | 12 | 9 | 26 | 95 | 17 | 2600 |
|Pittsburg| 24 | 14 | 17 | 13 | 28 | 99 | 20 | 2900 |
|**Demanda [Ton]**| 900 | 1200 | 600 | 400 | 1700 | 1100 | 1100 | $ \sum{6900}$ |

![Transporte](../Figuras/1.2.png)

## Formulação General

### Conjuntos
- $ S: \text{ Conjunto de siderúrgicas}$
- $ F: \text{Conjunto de fábricas}$

### Parâmetros
- $ c_{s,f} : \text{Custo de tranporte por tonelada da siderúrgica s para a fábrica f} $
- $ p_{s} : \text{Produção da siderúrgica en toneladas} $
- $ d_{f} : \text{Demanda da fábrica f em toneladas} $

### Variáveis de decisão

- $x_{s,f} : \text{Toneladas transportadas da siderúrgica s para a fábrica f}$ 

### Função objetivo 

- $ \mathrm{Min} \sum\limits_{s \in S} \sum\limits_{f \in F} c_{s,f} x _{s,f}$

### Restrições 

- $ \sum\limits_{f \in F} x_{s,f} = p_{s}  \;\;\;\;\;\; \forall s \in S $
- $ \sum\limits_{s \in S} x_{s,f} = d_{f} \;\;\;\;\;\; \forall f \in F $
- $ x_{s,f} \geq 0 \;\; \forall s \in S \text{, } f \in F $



In [47]:
import pandas as pd


### Leitura de dados

In [48]:
df = pd.read_excel('../Datos/1.2.xlsx','Datos', header=0, index_col=0)
print(df)

           Farmington  Detroit  Lansing  Winsor  St. Louis  Fremont  Lafayette
Gary               39       14       11      14         16       82          8
Cleveland          27        9       12       9         26       95         17
Pittsburg          24       14       17      13         28       99         20


In [49]:
df_2 = pd.read_excel('../Datos/1.2.xlsx','Prod', header=0, index_col=0)
print(df_2)

           Producao
Gary           1400
Cleveland      2600
Pittsburg      2900


In [50]:
df_3 = pd.read_excel('../Datos/1.2.xlsx','Dem', header=0, index_col=0)
print(df_3)

            Demanda
Farmington      900
Detroit        1200
Lansing         600
Winsor          400
St. Louis      1700
Fremont        1100
Lafayette      1000


### Conjuntos

In [51]:
SID = list(df.index.map(str))
FAB = list(df.columns.map(str))

### Parâmetros

In [52]:
costos = {(s,f):df.at[s,f] for s in SID for f in FAB}

In [53]:
producao = df_2.to_dict()['Producao']

In [54]:
demanda = df_3.to_dict()['Demanda']

### Modelamento no Pyomo

In [55]:
from pyomo.environ import *
model = ConcreteModel()

#### Conjuntos e Parâmetros

In [56]:
S = (SID)
F = (FAB)
c = (costos)
p = (producao)
d = (demanda)


In [57]:
model.x = Var(S,F, domain = NonNegativeReals)

In [58]:
def obj_rule(m):
    return sum(c[s,f]*m.x[s,f] for s in S for f in F)
model.obj = Objective(rule = obj_rule, sense = minimize)

In [59]:
def pro_cost_rule(m,s):
    return sum(m.x[s,f] for f in F) == p[s]
model.restricao_1 = Constraint(S, rule=pro_cost_rule)

In [60]:
def dem_cost_rule(m,f):
    return sum(m.x[s,f] for s in S) == d[f]
model.restricao_2 = Constraint(F, rule=dem_cost_rule)

In [67]:
Resultado = SolverFactory('cbc', executable='C:/Solvers/cbc.exe').solve(model)

In [75]:
model.x.pprint()

x : Size=21, Index=x_index
    Key                         : Lower : Value  : Upper : Fixed : Stale : Domain
       ('Cleveland', 'Detroit') :     0 : 1200.0 :  None : False : False : NonNegativeReals
    ('Cleveland', 'Farmington') :     0 :    0.0 :  None : False : False : NonNegativeReals
       ('Cleveland', 'Fremont') :     0 :    0.0 :  None : False : False : NonNegativeReals
     ('Cleveland', 'Lafayette') :     0 :  400.0 :  None : False : False : NonNegativeReals
       ('Cleveland', 'Lansing') :     0 :  600.0 :  None : False : False : NonNegativeReals
     ('Cleveland', 'St. Louis') :     0 :    0.0 :  None : False : False : NonNegativeReals
        ('Cleveland', 'Winsor') :     0 :  400.0 :  None : False : False : NonNegativeReals
            ('Gary', 'Detroit') :     0 :    0.0 :  None : False : False : NonNegativeReals
         ('Gary', 'Farmington') :     0 :    0.0 :  None : False : False : NonNegativeReals
            ('Gary', 'Fremont') :     0 : 1100.0 :  None : Fals

#### Parâmetros